In [ ]:
%matplotlib inline
from cg import Point, turn
from hidden import *       # Функции, реализация которых не особо интересна в свете этой темы
from answers import *      # Ответы
from visual_utils import * # Функции визуализации
from cg_common import *    # Должно быть перенесено в общую либу

# Ушной метод триангуляции многоугольников

В теме, посвященной [триангуляции монотонного многоугольника](../8_scan_line_triangulation/1_Monotone_Pieces.ipynb#Теорема 1 о существовании триангуляции многоугольника), мы доказали существование триангуляции у простого $n$-вершинного многоугольника и разобрали один из способов ее нахождения.
Здесь мы рассмотрим другой довольно интересный, а также простой в понимании и реализации алгоритм триангулирования многоугольников, называющийся **ушной триангуляцией**.  

### Определение
Три последовательные вершины $v_{i-1}$, $v_{i}$, $v_{i+1}$ многоугольника $P$ называются **ухом**, если треугольник, образованный этими вершинами, не содержит внутри себя и на границе остальных вершин многоугольника $P$, а так же угол при вершине $v_{i}$, называемой **вершиной уха**, строго меньше $\pi$ (см. рис. 1). 

![определение уха](./img/def_ear.svg)

Отметим, что мы будем допускать, чтобы остальные вершины многоугольника пересекались с ухом по его вершинам. Это необходимо для будущего обобщения на многоугольники с внутренними дырками. Когда мы затронем их, станет понятен смысл таких допущений.  

#### Теорема 1 (О существовании двух ушей у многоугольника)
> У любого простого $n$-вершинного многоугольника $P$ всегда существуют два непересекающихся между собой уха.

$\triangleright$
<div style="padding-left:40px">
Как было отмечено ранее, у любого простого $n$-вершинного многоугольника существует триангуляция. По ней можно построить двойственный граф: каждую исходную грань переводим в новую вершину, а новые вершины, соответствующие смежным по ребру граням, соединяем ребром (см. рис. 2). Заметим, что этот граф является деревом. Так как ребро, по которому исходные грани смежны, всегда является диагональю многоугольника, то двойственный граф распадется на две компоненты связности, если удалить из него соответствующее двойственное ребро. Это верно для всех ребер двойственного графа, поэтому он является деревом. У дерева есть хотя бы два листа, т.е. две грани триангуляции, являющиеся непересекающиеся ушами исходного многоугольника.      
![Не ухо](./img/2_ears_exist.svg)
</div>
$\triangleleft$

### Задача
>Пусть нам дается список вершин простого $n$-вершинного многоугольника, задающий их в порядке обхода против часовой стрелки. 
Надо триангулировать этот многоугольник.

### Алгоритм
Идея заключается в последовательном отрезании ушей от многоугольника, пока он не станет треугольником.
Зафиксируем стартовую вершину многоугольника и будем обходить его в поиске ушей, добавляя в стек $S$ вершины, обработанные нами и не являющиеся вершинами ушей.
Ломаная, построенная на вершинах из стека, будет отделять отрезанные уши от еще не просмотренных вершин многоугольника. 

Когда при обходе мы находим вершину уха, то отрезаем ее, добавляя диагональ, и переходим к предыдущей вершине из стека.
Если она стала вершиной уха, то отрезаем ее и дальше спускаемся по стеку.
Этот процесс остановится, когда мы либо достигнем стартовой вершины (стек пуст), либо текущая вершина перестанет быть ухом.
Тогда мы переходим к вершине, являющейся концом последней добавленной диагонали, и продолжаем обход многоугольника в поиске ушей.

В итоге, мы придем к тому, что от нерассмотренной части многоугольника останется только одно ребро, а остальная часть будет триангулирована.  

### Ушная проверка
Для того чтобы быть ухом, вершина прежде всего должна быть выпуклой, для проверки этого, удобно использовать [левый поворот](../5_affine_space/foundation.ipynb#Ориентация).
Затем нужно проверить, что никакие вершины многоугольника не лежат внутри или на границе треугольника, соответствующего текущей предполагаемой вершине уха.

Вам предлагается потренироваться в решении этой задачи, дополнив следующую функцию.

In [ ]:
def is_outer(v0,v1,v2,v):
    '''
    Проверяет, лежит ли вершина строго снаружи треугольника.
    
    На вход подаются три вершины v0, v1 и v2, образующие потенциальное ухо, а также
    вершина многоугольника, принадлежность которой надо проверить. Вершины имеют тип
    Point. У них есть поля x, y. Вам также доступна функция turn - предикат левого
    поворота, которая принимает три Point.
    
    Требуется вернуть True, если v строго снаружи треугольника, иначе False.
    '''
    
    return is_outer_answer(v0,v1,v2,v) # Заглушка с функцией-ответом на задачу

#### Лемма 1 (О связи вершин в стеке с потенциальными ушами)
> Во время ушной проверки на принадлежность уху достаточно тестировать только еще нерассмотренные вершины.

$\triangleright$
<div style="padding-left:40px">
Нерассмотренные вершины мы проверяем всегда, поэтому если хотя бы одна из них попала в треугольник, то рассмотренные ранее вершины (находящиеся в стеке) не влияют на результат ушной проверки.

Пусть в треугольник попали только рассмотренные вершины (см. рис. 3, красным отмечены ребра, построенные на вершинах из стека), выберем из них самую ближайшую к $v$ (вершина потенциального уха), назовем ее $w_{i}$.
Рассмотрим многоугольник, образованный вершинами $w_{i}, w_{i + 1},\dots, w_{k}, v, w_{i}$, где $w_{i}, w_{i + 1},\dots, w_{k}$ - непрерывная цепочка вершин в стеке.
По теореме 1 у него существуют два непересекающихся уха, так что даже если предположить, что вершиной одного из них является $v$ или $w_{i}$, то есть второе, которое соответствует вершине из стека $S$, а это противоречит инварианту стека, по которому вершины в $S$ не являются ушами.
![ear_test](img/lemma_1.svg)
</div>
$\triangleleft$

Теперь соберем полученные факты вместе и напишем полноценную ушную проверку.

In [ ]:
def is_ear(hedge, probable_ear, pred):
    '''
    Проверяет, образует ли треугольник ухо.
    
    На вход подается: экземпляр класса Hedge - ребро, соответствующее вершине, с которой надо 
    начинать проверку на принадлежность; массив из трех последовательных вершин, образующих 
    потенциальное ухо; предикат, задающий нерассмотренные вершины, который мы определим 
    немного позже в соответствии с леммой 1.
    
    Возвращает True, если probable_ear действительно образует ухо, иначе False.
    '''
    
    while pred(hedge):
        if not is_outer(*probable_ear, hedge.origin):
            return False 
        hedge = hedge.next
    return True 

А теперь давайте напишем сам алгоритм.
Воспользуемся фактом, что триангуляция $n$-вершинного многоугольника содержит $n - 2$ треугольника (это доказано в теме монотонной триангуляции многоугольника), тогда каждое отрезанное ухо порождает диагональ, а последняя такая образует сразу два треугольника, поэтому в итоге $n - 3$ диагонали. Это будет признаком завершения алгоритма.

In [ ]:
# Примеры различных многоугольников
examples = {
        'some': [(4, 0), (2, 1), (3, 6), (0, 5), (-1, 6), (-3, 4), (-1, 2), (-2, 0),
              (-4, 1), (-5, -2), (-2, -4), (0, -3), (2, -4), (1, -1), (3, -2)]
        ,
        'square': [(0,0), (1,0), (1,1), (0,1)]
        ,
        'arrow': [(0,0), (2,3), (4,0), (2,6)]
        ,
        'star': [(0,0), (7,1), (7,3), (6,5), (5,6), (3,7), (1,7)]
        ,
        'substar': [(0,0), (1,6), (7,1), (7,3), (6,5), (5,6), (3,7), (1,7)]
        ,
        'hook': [(0,3), (2,4), (3,3), (3,2), (2,1), (2,2), (2,3), (1,2), (1,1), (3,0), (4,4), (2,5), (1,4), (0,5)]
        }

# Вершины многоугольника в порядке обхода против часовой стрелки
source = examples['hook']
# Преобразовываем пары в наш класс
# source = list(map(lambda i: Point(i[0], i[1]), source))
source = [Point(p[0], p[1]) for p in source]

In [ ]:
def break_into_pieces_triang(D, folder='.ear_clipping'):
    # Стек вершин
    S = []
    # Добавленные диагонали будем хранить в отдельном списке для наглядной визуализации
    D1 = []
    
    # С помощью этой функции мы будем делать "снимки" состояния алгоритма, чтобы потом
    # можно было посмотреть его по шагам.
    dump = create_dump_func(folder, visual_dump_ear_clipping_triangulation, D, D1, S)
    
    t = 0
    start = D[0]
    dump(start.origin)
    S.append(start)    
    
    while len(D1) < len(D) - 3:
        cur = S[-1].next
        while True:
            # Вернулись в стартовую вершину 
            if len(S) == 0:
                dump(cur.origin)
                if len(D1) >= len(D) - 3:
                    break
                S.append(cur)
                cur = cur.next
            
            # Рассматриваем потенциальное ухо
            v0 = S[-1].origin
            v1 = cur.origin
            v2 = cur.next.origin
            
            if turn(v0, v1, v2) >= 1 and is_ear(cur.next.next, probable_ear=[v0, v1, v2],
                                                pred=lambda hedge: hedge != start):
                dump(cur.origin)
                # Спускаемся по стеку 
                prev = S.pop()                
                d = add_diagonal_with_next(prev, cur.next)
                D1.append(d)
                dump(cur.origin)

                cur = d
            else:
                # Идем дальше по многоугольнику
                dump(cur.origin)
                S.append(cur)
                cur = cur.next
                break
                
    # Удаляем лишнее ребро, образованное удалением последнего уха
    S.clear()
    dump(None)

In [ ]:
# Запуск алгоритма
# Многоугольник хранится в DCEL
break_into_pieces_triang(build_dcel(source))

Можно проследить за алгоритмом по шагам, используя виджет ниже.
У исходного многоугольника рёбра чёрные.
Рёбра, образованные вершинами из стека и текущей вершиной - красные.
А диагонали разбиения проведены синим пунктиром.
Желтая точка является текущей вершиной.

In [ ]:
from IPython.display import display
display(SlideShower('.ear_clipping'))

#### Доказательство корректности
Доказательство корректности алгоритма достаточно легко вывести из его построения.
Отрезаемые уши попарно не пересекаются, а следовательно и диагонали попарно не пересекаются.
Отрезая ухо, мы добавляем диагональ, которая сохраняет корректность многоугольника, при этом уменьшая количество вершин в нем.
В итоге многоугольник становится треугольником, который по определению триангулирован, а все вершины соединены диагоналями.

#### Оценка времени работы
Каждая вершина бывает в стеке только один раз.
На каждом шаге алгоритма мы либо отрезаем ухо и спускаемся по стеку, либо переходим к следующей вершине многоугольника.
Для каждой добавленной вершины в начинающейся с нее цепочке последовательных отрезаний ушей мы можем зафейлить ушную проверку только один раз, после чего добавим следующую вершину полигона в стек.
Т.е. полный такой обход полигона занимает $O(n)$ времени.
Ушная проверка для одной вершины занимает $O(n)$ времени.
А тогда общая оценка на время $O(n^2)$.

#### Оценка на память
Для хранения самого полигона мы используем DCEL, который занимает линейное количество памяти.
И заводим стек для хранения диагоналей, которых $n-2$.
Таким образом, потребляемое количество памяти линейно.

Итого, мы получили следующий результат:
> Простой многоугольник с $n$ вершинами может быть триангулирован за $O(n^2)$ времени, используя $O(n)$ памяти. 

### Триангуляция многоугольника с дырками

Две вершины называются **взаимно видимыми**, если соединяющая их диагональ не пересекает ребра многоугольника и не содержит вершин из него.

![Взаимно видимые вершины](img/def_visible.svg)

Пусть теперь мы хотим триангулировать полигон, в котором есть дырки.
Для начала предположим, что она одна.
Сведем задачу к триангуляции без дырок.
Идея заключается в том, чтобы соединить **взаимно видимые** вершину внешнего и вершину внутреннего (задающего дырку) полигонов.
Пусть вершина $v$ принадлежит внешнему, а $u$ - внутреннему (см. рис. 4).
Добавив две новые вершины $v'$ и $u'$, ребра $v'u'$ (для перехода от внешнего полигона к дырке) и $uv$ (для перехода от дырки ко внешнему полигону), сделаем наш многоугольник простым.
А его мы уже умеем триангулировать.

Легко понять, что научившись сливать одну дырку и задавать порядок это процесса для остальных, мы сможем исключить любое количество дырок из многоугольника.

Однако как мы найдем взаимно видимые вершины? 

### Алгоритм поиска взаимно видимых вершин
Найдем самую левую точку внутреннего полигона, а из таких самую нижнюю, назовем ее $p$.
Проведем из нее луч в минус бесконечность по оси $Ox$.
Он пересечет какие-то ребра или вершины внешнего полигона.
Возьмем из таких точек пересечения самую правую, назовем ее $c$.
Возможны два случая:
* Если это вершина, то мы нашли подходящую видимую вершину внешнего полигона (см. рис. 5б).
* Иначе мы попали во внутреннюю область ребра $e'e$ (см. рис. 5а).
Тогда соединим его нижний конец $e$ и $p$.
Рассмотрим невыпуклые вершины внешнего полигона, которые попали в треугольник $\Delta epc$.
Отсортируем их по углу, образованному с отрезком $pc$.
Возьмем минимальную, а среди таких самую правую.

![рис 7](img/algo_hole.svg)

Докажем сразу корректность данного метода, а потом займемся его реализацией.

### Корректность алгоритма 
Выбрав самую левую и нижнюю вершину дырки, мы гарантируем, что проведенный из нее луч пересечет только вершины и ребра внешнего полигона.
Очевидно, что кандидатом на взаимную видимость является элемент, с самым правым пересечением, так как он сам является помехой для всех остальным.
Если мы попали в вершину, то так как правее нее ничего не пересекает отрезок видимости, то она является взаимно видимой с вершиной дырки.
Если мы попали во внутренность ребра, то спускаемся до его нижнего конца.
Заметим, что отрезок, соединяющий этот край и точку из дырки, легко мог пересечь некоторые ребра внешнего полигона.
Однако это единственная сторона треугольника, через которую вершины могли попасть внутрь.
Отсортировав их по углу и координате оси $Ox$ и выбрав минимальную, можно утверждать, что она является искомой по построению.

Из алгоритма так же ясно, что дырки надо сливать в лексикографическом порядке их самых левых нижних точек.

Теперь займемся реализацией описанного алгоритма.

Для детального понимания того, как добавляются ребра, связывающие внешний и внутренний полигоны, можете обратиться к файлу hidden.py. Здесь же предлагаемся посмотреть на обход конечного полигона.

In [ ]:
# Внешний многоугольник
polygon = list(map(lambda i: Point(i[0], i[1]),
        [(3, 1), (0, 3), (-4, 2), (-2, 0), (-3, -1), (-2, -7), (1, -3), (3, -4), (4, -1)]))
# Точки "дырок" мы задаём уже по часовой стрелке - многоугольник снаружи них, а не внутри
hole1 = list(map(lambda i: Point(i[0], i[1]),
        [(0, 1), (1, 0), (0, -2), (-1, 0)])) 
hole2 = list(map(lambda i: Point(i[0], i[1]),
        [(-1, -2), (-1, -5), (-2, -3)]))

In [ ]:
outer = build_dcel(polygon)
inner1 = build_dcel(hole1)
inner2 = build_dcel(hole2)

Функция $\operatorname{merge\_polygons}$ принимает первым аргументом внешний полигон, затем его взаимно видимую вершину, третьим аргументов является дырка, а четвертым ее взаимно видимую вершину.

In [ ]:
# В этом примере пара взаимно видимых точек заранее подобрана.  
D = merge_polygons(outer, outer[0], inner1, inner1[0])
D = merge_polygons(outer, outer[4], inner2, inner2[0])

In [ ]:
def break_into_pieces_bypass_forward(D, folder='.bypass_forward'):
    dump = create_dump_func(folder, visual_dump_bypass, D)
    
    start = D[0]
    dump(start.origin)
    cur = start.next
    while cur != start:
        dump(cur.origin)
        cur = cur.next
        
break_into_pieces_bypass_forward(D)

from IPython.display import display
display(SlideShower('.bypass_forward'))

Для алгоритма зададим более показательный пример многоугольника.

In [ ]:
# Внешний многоугольник
polygon = list(map(lambda i: Point(i[0], i[1]),
        [(8, 18), (2, 12), (1, 14), (2, 11), (6, 15), (3, 3), (5, 1), (4, 5),(5, 6),
         (6, 8), (8, 4), (8, 7), (7, 8), (9, 9), (9, 10), (13, 8), (18, 13), (17, 15)]))
# Точки "дырок" мы задаём уже по часовой стрелке - многоугольник снаружи них, а не внутри
hole1 = list(map(lambda i: Point(i[0], i[1]),
       [(15, 13), (14, 10), (12, 12), (14, 13), (12, 14), (15, 15)])) 
hole2 = list(map(lambda i: Point(i[0], i[1]),
        [(11, 16), (9, 13), (8, 15)]))

Напишем класс, которым воспользуемся для сортировки невыпуклых вершин внутри треугольника.
Левая нижняя точка из дырки будет вершиной угла (одинаковой для всех).
Один луч будет направлен влево по оси Ох.
А второй будет задавать невыпуклая вершина.
Величину угла будем определять уже упомянутым предикатом левого поворота.

In [ ]:
class Angle:
    def __init__(self, vertex, angular_point):
        self.vect = vertex
        self.angular_point = angular_point 
        
    def __lt__(self, other):
        t = turn(self.vect, self.angular_point, other.vect) 
        return t < 0 or t == 0 and self.vect.x > other.vect.x

Соберем все кусочки вместе.

In [ ]:
def break_into_pieces_holes_merging(polygon, *holes, folder='.holes_merging'):
    # converter to Point
    tp = lambda x: Point(int(x[0]), int(x[1]))
    # converter to np.array
    tnp = lambda v: np.array([v.x, v.y]) 

    # Вместо луча в минус бесконечноть по оси Ох будем проводить
    # отрезок до координаты самой левой точки полигона.
    min_x = min(polygon, key = lambda h: h.origin.x).origin.x
   
    # Отсортируем дырки по неубыванию самой левой нижней точки
    order = []
    for i, hole in enumerate(holes):
        # left_lower_hedge
        llh = hole[0]
        for h in hole:
            if h.origin < llh.origin:
                llh = h
        order.append((i, llh))

    ordered_holes = []
    for pos, llh in sorted(order, key=lambda v: v[1].origin):
        ordered_holes.append((llh, holes[pos]))
    
    # С помощью этой функции мы будем делать "снимки" состояния алгоритма, чтобы потом
    # можно было посмотреть его по шагам.
    dump = create_dump_func(folder, visual_dump_holes_merging,
                            polygon, list(map(lambda x: x[1], ordered_holes)))
    
    dump({'step':Merging_step.BEFORE_START})
    
    # По очереди сливаем дырки с полигоном
    for llh, hole in ordered_holes:
        dump({'llh':llh,'step':Merging_step.LEFT_LOWER})
        
        # Ищем ребро или вершину с максимальным по оси Ох пересечением 
        intersected = []
        a = llh.origin
        b = tp([min_x, llh.origin.y])
        
        dump({'ray':(a,b), 'step':Merging_step.RAY})
        for h in polygon:
            c = h.origin
            d = h.twin.origin
            if turn(a, b, c) * turn(a, b, d) <= 0 and turn(c, d, a) * turn(c, d, b) <= 0:
                ip = get_intersection_point(tnp(a), tnp(b), tnp(c), tnp(d))
                intersected.append((ip[0]/ip[2], ip[1]/ip[2], h))
        
        # Сортируем точки пересечения по убыванию координаты x
        intersected = sorted(intersected, key=lambda x: -x[0])
        dump({'ray':(a,b), 'intersected':intersected, 'step':Merging_step.INTERSECTED})
        
        # Выбираем отрезок\вершину с ближайшей к вершине дырки точкой пересечения 
        closest = intersected[0]
        ip = (closest[0], closest[1])
        v0 = a
        v1 = closest[2].origin
        v2 = closest[2].twin.origin
        
        if v1.y != v2.y and turn(a, b, v1) == 0:
            # Попали в начало ребра
            dump({'ray':(a,b), 'closest':v1, 'step':Merging_step.REFLEX_VERTEXES})
            merge_polygons(polygon, closest[2], hole, llh)
            dump({'polygon':polygon, 'step':Merging_step.MERGE})
            continue
        elif v1.y == v2.y or turn(a, b, v2) == 0:
            # Попали в конец ребра(или оно горизонтальное)
            dump({'ray':(a,b), 'closest':v2, 'step':Merging_step.REFLEX_VERTEXES})
            merge_polygons(polygon, closest[2].next, hole, llh)
            dump({'polygon':polygon, 'step':Merging_step.MERGE})
            continue
        
        # Попали в ребро
        dump({'ray':(a,b), 'closest':closest[2], 'step':Merging_step.REFLEX_VERTEXES})
        
        # Ищем выпуклые вершины, попавшие в треугольник
        reflex_origins = [closest[2].next] # нижняя вершина ребра, которое пересек луч
        for h in polygon:
            v = h.origin
            # surf_intersection(p: Point, *points)
            # Определяем принадлежность треугольнику по прямым, 
            # на которых лежат его стороны, а не по самим сторонам 
            if turn(h.prev.origin, v, h.next.origin) <= 0 and v != v2 and \
                surf_intersection(v, (llh.origin, b), (v1, v2), (v2, llh.origin)):
                reflex_origins.append(h)
        dump({'triangle':(v0,ip,v2), 'reflex':reflex_origins, 'step':Merging_step.REFLEX_SORTING})
        
        # Отсортируем вершины по углу относительно отрезка ab
        reflex_origins = sorted(reflex_origins, key=lambda hedge: Angle(hedge.origin, llh.origin))
        dump({'triangle':(v0,ip,v2), 'closest':reflex_origins[0], 'step':Merging_step.CLOSEST})

        merge_polygons(polygon, reflex_origins[0], hole, llh)
        dump({'polygon':polygon, 'step':Merging_step.MERGE})
        

In [ ]:
# Запуск алгоритма
D = build_dcel(polygon)
break_into_pieces_holes_merging(D,
                                build_dcel(hole1),
                                build_dcel(hole2))

Можно проследить за алгоритмом по шагам, используя виджет ниже.
У исходного многоугольника и дырок чёрные рёбра.
Желтой точкой отмечена самая левая и нижняя вершина дырки.
Красными отмечены ребра, которые пересек луч, а так же потом и самое правое из них.
Красными точками отмечен потенциальные взаимно видимые вершины внешнего полигона, а так же итоговая выбранная.
Фиолетовым пунктиром показаны границы треугольника, в котором ищем невыпуклые вершины, а синим пунктиром - углы к ним.

In [ ]:
from IPython.display import display
display(SlideShower('.holes_merging'))

Триангулируем полученный многоугольник.

In [ ]:
break_into_pieces_triang(D)

In [ ]:
from IPython.display import display
display(SlideShower('.ear_clipping'))